<h1 style="display:inline-block">ActiveVoxel Design</h1><br/>

subdivide( get_faces( solid ), divisions )

In [ ]:
cube_vxs = [[ px py pz id ] ...]
            

In [13]:
import numpy as np

vxs_py = [[ 1, 0, 0, 0 ], [ 0, 0, 0, 1 ], [ 0, 0, 1, 2 ], [ 1, 0, 1, 3 ], [ 0, 1, 1, 4 ], [ 1, 1, 1, 5 ], 
       [ 0, 1, 0, 6 ], [ 1, 1, 0, 7 ]]

vxs = np.array( vxs_py )

print( vxs.transpose() )


[[1 0 0 1 0 1 0 1]
 [0 0 0 0 1 1 1 1]
 [0 0 1 1 1 1 0 0]
 [0 1 2 3 4 5 6 7]]


In [15]:
tmax = 1

vx_names = { ( tmax + 1 ) * ( tmax + 1 ) * vx[ 0 ] + ( tmax + 1 ) * vx[ 1 ] + vx[ 2 ]: 
             ( vx[ 3 ], vx[ 0 ], vx[ 1 ], vx[ 2 ], tmax ) for vx in vxs_py }

print( vx_names )


{4: (0, 1, 0, 0, 1), 0: (1, 0, 0, 0, 1), 1: (2, 0, 0, 1, 1), 5: (3, 1, 0, 1, 1), 3: (4, 0, 1, 1, 1), 7: (5, 1, 1, 1, 1), 2: (6, 0, 1, 0, 1), 6: (7, 1, 1, 0, 1)}
